# **Rocket Experiments**

## **Initialisation**

In [ ]:
pip install sktime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.9/28.9 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.1/134.1 kB 5.1 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import pickle
import itertools
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sktime.transformations.panel.rocket import MiniRocketMultivariate
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from tensorflow.keras.regularizers import l2
from keras.layers import Input, Layer, Conv1D, MaxPool1D, ReLU, BatchNormalization, LayerNormalization, Dropout, Add, Dense, GlobalMaxPooling1D, Bidirectional, GRU

In [ ]:
#You MUST run this command before reading in any data from Google Drive
from google.colab import files
from google.colab import drive
import pandas as pd
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/My Drive/Colab Notebooks/Thesis/experiments')

%run ../sys_configs.ipynb

Mounted at /content/drive


In Convolutional Neural Networks (CNNs), the convolutional filters that form the backbone of the network give rise to a large number of trainable parameters compared to algorithms like ridge regression. The Random Convolutional Kernel Transform (Rocket) was built to address this, and it performs feature extraction by overlaying a large number of random convolutional kernels, and then extracting features from the outputs of these kernels. Rocket therefore has the effect of transforming the problem to one that standard ML algorithms like Random Forest and Ridge Regression can solve. The original Rocket architecture computed 2 such features: the maximum value and the Proportion of Positive Values (PPV). Any individual feature can be thought of as a *base learner* featuring only a weak signal, but the base learners are followed up with a classifier like a Ridge Classifier that selects the most relevant features. This approach has a much lower computational complexity compared to traditional time series classification algorithms, yet in the benchmark scripts was found to have the strongest performance of the models tested on MyoGym data.

MultiRocket extends on the Rocket and MiniRocket architectures (covered in another benchmark workbook). In MultiRocket, 84 convolutional kernels are applied to both the raw time series and the first order differenced time series, which matches the number of kernels used in MiniRocket. Different dilation factors are used for each kernel in order to extract features at different scales. Biases are sampled from the convolution outputs, or for Rocket, sampled from a Uniform distribution in the range $(-1, 1)$.  MultiRocket features 4 pooling operators: Proportion of Positive Values (PPV), Mean of Positive Values (MPV), Mean of Indices of Positive Values (MIPV) and Longest Stretch of Positive Values (LSPV).

In [ ]:
with open('../data/1s_train.npy', 'rb') as f:
    x1s_train = np.load(f)
    y1s_train = np.load(f)
N, sz, dim = x1s_train.shape

with open('../data/1s_val.npy', 'rb') as f:
    x1s_val = np.load(f)
    y1s_val = np.load(f)

In [ ]:
# Convert the labels to tensors
train_labels_tf = tf.one_hot(y1s_train, 31, dtype=tf.float32)
val_labels_tf = tf.one_hot(y1s_val, 31, dtype=tf.float32)

#### **Prepare standard train & validation datasets**

In [ ]:
# Convert the dataset to tensors
train_data_tf = tf.convert_to_tensor(x1s_train, dtype=tf.float32)
val_data_tf = tf.convert_to_tensor(x1s_val, dtype=tf.float32)

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((train_data_tf, train_labels_tf))
val_ds = tf.data.Dataset.from_tensor_slices((val_data_tf, val_labels_tf))

train_ds = train_ds.shuffle(500)

train_ds = train_ds.padded_batch(64)
val_ds = val_ds.padded_batch(64)

#### **Prepare Rocket transformation train & validation datasets**

In [ ]:
# Transpose the train and validation data as the format needs to be N x D x T
x1s_train_ = x1s_train.transpose((0, 2, 1))
x1s_val_ = x1s_val.transpose((0, 2, 1))

# Compute the MiniRocket transform and transform to tensors
minirocket_multi = MiniRocketMultivariate(num_kernels = 10000, max_dilations_per_kernel = 32)
minirocket_multi.fit(x1s_train_)

train_rocket_np = minirocket_multi.transform(x1s_train_).to_numpy()
val_rocket_np = minirocket_multi.transform(x1s_val_).to_numpy()

train_rocket_tf = tf.convert_to_tensor(train_rocket_np, dtype = tf.float32)
val_rocket_tf = tf.convert_to_tensor(val_rocket_np, dtype = tf.float32)


In [ ]:
train_rocket_ds = tf.data.Dataset.from_tensor_slices((train_rocket_tf, train_labels_tf))
val_rocket_ds = tf.data.Dataset.from_tensor_slices((val_rocket_tf, val_labels_tf))

train_rocket_ds = train_rocket_ds.shuffle(500)

train_rocket_ds = train_rocket_ds.padded_batch(64)
val_rocket_ds = val_rocket_ds.padded_batch(64)

In [ ]:
C = len(set(y1s_train)) # Number of classes

#### **Create combined dataset for GridSearchCV**

In [ ]:
x1s = np.concatenate((x1s_train, x1s_val), axis = 0)
y1s = np.concatenate((y1s_train, y1s_val), axis = 0)

x1s_ = x1s.transpose((0, 2, 1))

minirocket_multi2 = MiniRocketMultivariate(num_kernels = 10000, max_dilations_per_kernel = 32)
minirocket_multi2.fit(x1s_)
rocket_np = minirocket_multi2.transform(x1s_).to_numpy()


In [ ]:
# Scramble the dataset
reordered_idxs = np.random.choice(len(rocket_np), size = len(rocket_np), replace = False)
rocket_np = rocket_np[reordered_idxs.copy()]
y1s = y1s[reordered_idxs.copy()]

## **Experiment: Customised ROCKET transformation**

All ROCKET-based models share the property that the convolutional weights are non-trainable. In MiniROCKET and MultiRocket, the convolutions are constrained to be of length 9 and to have 6 values of *-1* and 3 values of *+2*. Below, we define the kernels which will be used in ROCKET.

In [ ]:
exemplar_kernel = np.array([-1, -1, -1, -1, -1, -1, 2, 2, 2]) # 6 elements are -1, 3 elements are +2

# Find all permutations of that exemplar kernel
kernels = list(itertools.permutations(exemplar_kernel))
kernels = np.unique(np.array(kernels), axis = 0)
print(f"There are 9C6 = 84 kernels. Here are the first 10:\n{kernels[:10]}")

There are 9C6 = 84 kernels. Here are the first 10:
[[-1 -1 -1 -1 -1 -1  2  2  2]
 [-1 -1 -1 -1 -1  2 -1  2  2]
 [-1 -1 -1 -1 -1  2  2 -1  2]
 [-1 -1 -1 -1 -1  2  2  2 -1]
 [-1 -1 -1 -1  2 -1 -1  2  2]
 [-1 -1 -1 -1  2 -1  2 -1  2]
 [-1 -1 -1 -1  2 -1  2  2 -1]
 [-1 -1 -1 -1  2  2 -1 -1  2]
 [-1 -1 -1 -1  2  2 -1  2 -1]
 [-1 -1 -1 -1  2  2  2 -1 -1]]


In [ ]:
w = np.expand_dims(kernels, axis = 2)
w = np.tile(w, reps = (1,1,6))
w = np.transpose(w, (1, 2, 0))
print(f"The shape of the convolutional filter weights is {w.shape}")

The shape of the convolutional filter weights is (9, 6, 84)


The biases can be defined using the quantiles from a randomly chosen sample in the batch (as in MiniROCKET and MultiROCKET) or using a uniform distribution in the range $(-1, 1)$ (as in the original ROCKET paper). The MiniROCKET and MultiROCKET architectures recycle the 84 defined kernels with `num_dilation_factors = 32` different dilation factors, which are equidistant in the log space. Below, we define a custom Rocket layer. After overlaying each kernel at each dilation factor, we compute the Proportion of Positive Values (PPV), Mean of Positive Values (MPV) and Mean Index of Positive Values (MIPV).


In [ ]:
class Rocket(Layer):

    @staticmethod
    def ppv(tensor):
        """
        Compute the proportion of positive values in a tensor.
        """
        mask_ = tf.cast(tensor > 0, tf.float32)
        ppv = tf.reduce_sum(mask_, axis = 1)/sz
        return tf.cast(ppv, tf.float32)

    @staticmethod
    def mpv(tensor):
        """
        Compute the mean of positive values in a tensor.
        """
        mask = tensor > 0
        mask_ = tf.cast(mask, tf.float32)

        sum_pos_vals = tf.reduce_sum(tf.where(mask, tensor, 0), axis = 1)
        count_pos_vals = tf.reduce_sum(mask_, axis = 1)
        mpv = sum_pos_vals/count_pos_vals
        mpv = tf.where(tf.math.is_nan(mpv), tf.zeros_like(mpv), mpv)
        return tf.cast(mpv, tf.float32)

    @staticmethod
    def mipv(tensor):
        """
        Compute the mean of indices of positive values in a tensor.
        """
        mask = tensor > 0
        mask_ = tf.cast(mask, tf.int32)

        indices = np.arange(mask.shape[1])
        weighted_indices = mask_ * indices[:, np.newaxis]
        positive_counts = tf.reduce_sum(mask_, axis=1)
        total_positive_indices = tf.reduce_sum(weighted_indices, axis=1)
        mipv = total_positive_indices / positive_counts
        mipv = tf.where(tf.math.is_nan(mipv), tf.zeros_like(mipv), mipv)
        return tf.cast(mipv, tf.float32)

    def __init__(self, num_features = 10000, max_dilations_per_kernel = 32, **kwargs):
        super().__init__(**kwargs)

        # Define the Rocket kernel weights
        self.w = tf.constant(w, dtype = tf.float32, name = "weights")

        # We seek to produce num_features = 10,000 (by default) features, and no more than 32 unique dilations
        self.num_features = num_features
        self.max_dilations_per_kernel = max_dilations_per_kernel

        # There are 84 kernels of size 9 by design
        self.num_kernels = 84
        self.kernel_sz = 9

        # List of the convolutions in the Rocket
        self.convs = []

    def fit_dilations(self, input_length):
        """
        A key step in the preparation of Mini-ROCKET is the construction of different dilation factors in the
        log space given in the paper.

        Smaller dilation factors ~ 1 are more common due to the log spacing. Therefore, more of the generated
        features will have smaller dilation factors.

        The outputs are the dilations and number of features per dilation for each kernel.
        """

        #
        # Credit code to: https://github.com/angus924/minirocket/blob/main/code/minirocket.py
        #

        # Each kernel can have no more than max_dilations_per_kernel, yet we need num_features_per_kernel features. Hence the multiplier.
        num_features_per_kernel = (self.num_features /3 ) // self.num_kernels
        true_max_dilations_per_kernel = min(num_features_per_kernel, self.max_dilations_per_kernel)
        multiplier = num_features_per_kernel / true_max_dilations_per_kernel

        # The maximum exponent is specified in the paper as below.
        max_exponent = np.log2((input_length - 1) / (9 - 1))

        # The dilations are distributed uniformly in the log space. We take the floor of the dilation (which
        # must be a whole number), then, because these values will now no longer be unique, count up how many
        # features there are per dilation.
        unique_dilations, num_features_per_dilation = \
            np.unique(np.logspace(0, max_exponent, true_max_dilations_per_kernel, base=2).astype(np.int32),
                      return_counts=True)

        # The values in num_features_per_dilation sum to  true_max_dilations_per_kernel, but they need to sum to
        # num_features_per_kernel. We apply the multiplier.
        num_features_per_dilation = (num_features_per_dilation * multiplier).astype(np.int32)  # this is a vector

        # We took the floor of the counts at 2 stages above, introducing a discrepancy/remainder which is now fixed
        remainder = num_features_per_kernel - np.sum(num_features_per_dilation)
        i = 0
        while remainder > 0:
            num_features_per_dilation[i] += 1
            remainder -= 1
            i = (i + 1) % len(num_features_per_dilation)

        return unique_dilations, num_features_per_dilation

    def build(self, input_shape):
        dilations, num_features_per_dilation = self.fit_dilations(input_shape[1])

        # Due to the log scaling, some dilations are repeated more than others.
        self.all_dilations = np.repeat(dilations, num_features_per_dilation)

        for rate in self.all_dilations:
            # Build the convolution for each dilation
            conv = Conv1D(filters = self.num_kernels,
                          kernel_size = self.kernel_sz,
                          strides = 1,
                          padding = "same",
                          dilation_rate = int(rate),
                         )

            # Original Rocket paper approach
            self.b = tf.random.uniform(shape = (self.num_kernels,),
                                       minval = -1,
                                       maxval = 1)

            conv.build((None, *input_shape[1:]))
            conv.set_weights([self.w, self.b])
            conv.trainable = False # Force the convolution layer to be non-trainable.
            self.convs.append(conv)


    def call(self, inputs):
        conv_outputs = []

        # Apply the convolution to each dilation in turn, then concatenate the outputs
        for conv in self.convs:
            conv_outputs.append(conv(inputs))
        concatenated = tf.concat(conv_outputs, axis=-1)


        ppv = Rocket.ppv(tensor = concatenated)
        mpv = Rocket.mpv(tensor = concatenated)
        mipv = Rocket.mipv(tensor = concatenated)
        #pool = self.max_pool(concatenated)

        output = tf.concat([ppv, mpv, mipv], axis = 1) #, pool], axis = 1)
        return output

## **Experiment: Rocket transformation with custom layer (bias from Uniform distribution)**

Let's first try to build a model similar to the *sktime* implementation of the MiniRocket using our custom Rocket. We expect similar performance with the custom Rocket to the sktime implementation (which was based off that produced in the original paper).

### **Without L2 loss penalty**

In [ ]:
def ReperformedRocket(shape):
    block1_input_layer = Input(shape=shape)
    z = Rocket(name = "Rocket1")(block1_input_layer)
    output_layer = Dense(C, activation="softmax")(z)
    return Model(inputs=block1_input_layer, outputs=output_layer)

In [ ]:
ReperformedRocket_model = ReperformedRocket(shape = (sz, dim))
ReperformedRocket_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 500, 6)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Rocket1 (Rocket)                     │ (None, 9828)                │         180,180 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 31)                  │         304,699 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 484,879 (1.85 MB)

 Trainable params: 304,699 (1.16 MB)

 Non-trainable params: 180,180 (703.83 KB)

In [ ]:
ReperformedRocket_model = ReperformedRocket(shape = (sz, dim))
ReperformedRocket_model.compile(optimizer=Adam(learning_rate=0.1, beta_1=0.99, beta_2=0.999, epsilon=1e-08), loss='categorical_crossentropy', metrics=['accuracy'])
history_rr = ReperformedRocket_model.fit(train_ds, validation_data=val_ds, epochs=100, verbose = 1)

Epoch 1/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 24s 302ms/step - accuracy: 0.0361 - loss: 311354.8750 - val_accuracy: 0.0242 - val_loss: 515379.7500
Epoch 2/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 24s 88ms/step - accuracy: 0.0421 - loss: 453903.3438 - val_accuracy: 0.0373 - val_loss: 308108.9062
Epoch 3/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 4s 94ms/step - accuracy: 0.0600 - loss: 302455.5938 - val_accuracy: 0.0484 - val_loss: 263539.1562
Epoch 4/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 4s 90ms/step - accuracy: 0.0572 - loss: 243898.2031 - val_accuracy: 0.0625 - val_loss: 104577.4844
Epoch 5/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 5s 95ms/step - accuracy: 0.0874 - loss: 110551.8594 - val_accuracy: 0.0696 - val_loss: 104429.8906
Epoch 6/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 5s 99ms/step - accuracy: 0.0866 - loss: 111190.0312 - val_accuracy: 0.1028 - val_loss: 88913.7969
Epoch 7/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 4s 93ms/step - accuracy: 0.1027 - loss: 71896.6328 - val_accuracy: 0.1845 - val_loss: 59936.2227
Epoch 8/100
48/48 ━━━━━━━━━━━━━━━━

The performance was competitive with most benchmark methods, but not with the sktime implementation. There were some significant differences between this custom implementation and the sktime implementation, notably that in the custom implementation padding was *always* applied, and the bias was sampled from the $Uniform(-1, 1)$ distribution rather than from a quantiles of a randomly selected training example.

### **With L2 loss penalty**

The experiment above was repeated with the following regularisation penalty applied in the fully connected layer: `output_layer = Dense(C, activation="softmax", kernel_regularizer = l2(10))(z)`. The parameter was set to 10 as this was the best hyperparameter found by the Cross Validation approach in the benchmark paper. Also tested were 0.1 and 1. All values of the hyperparameter resulted in substantially worse performance.

In [ ]:
def ReperformedRocketL1(shape):
    block1_input_layer = Input(shape=shape)
    z = Rocket(name = "Rocket1")(block1_input_layer)
    output_layer = Dense(C, activation="softmax", kernel_regularizer = l2(10))(z)
    return Model(inputs=block1_input_layer, outputs=output_layer)

In [ ]:
ReperformedRocket_model_L1 = ReperformedRocketL1(shape = (sz, dim))
ReperformedRocket_model_L1.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 500, 6)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Rocket1 (Rocket)                     │ (None, 29988)               │         549,780 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 31)                  │         929,659 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,479,439 (5.64 MB)

 Trainable params: 929,659 (3.55 MB)

 Non-trainable params: 549,780 (2.10 MB)

In [ ]:
ReperformedRocket_model_L1 = ReperformedRocketL1(shape = (sz, dim))
ReperformedRocket_model_L1.compile(optimizer=Adam(learning_rate=1.0, beta_1=0.99, beta_2=0.999, epsilon=1e-08), loss='categorical_crossentropy', metrics=['accuracy'])
history_rrl1 = ReperformedRocket_model_L1.fit(train_ds, validation_data=val_ds, epochs=50, verbose = 1)

Epoch 1/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 36s 504ms/step - accuracy: 0.0401 - loss: 5489845.0000 - val_accuracy: 0.0484 - val_loss: 4808758.5000
Epoch 2/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 10s 210ms/step - accuracy: 0.0394 - loss: 4160079.0000 - val_accuracy: 0.0383 - val_loss: 3883532.2500
Epoch 3/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 10s 209ms/step - accuracy: 0.0395 - loss: 3568623.2500 - val_accuracy: 0.0232 - val_loss: 3121545.5000
Epoch 4/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 10s 210ms/step - accuracy: 0.0338 - loss: 2925710.0000 - val_accuracy: 0.0474 - val_loss: 2478184.2500
Epoch 5/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 10s 212ms/step - accuracy: 0.0379 - loss: 2592972.0000 - val_accuracy: 0.0474 - val_loss: 2475056.0000
Epoch 6/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 10s 212ms/step - accuracy: 0.0307 - loss: 2309542.7500 - val_accuracy: 0.0242 - val_loss: 1929538.5000
Epoch 7/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 10s 213ms/step - accuracy: 0.0264 - loss: 1881276.8750 - val_accuracy: 0.0323 - val_loss: 1647636.0000
Epoch 8/50
 8

## **Experiment: Rocket transformation with *sktime* implementation**

### **Without L2 loss penalty**

In [ ]:
def RocketSktime(shape):
    block1_input_layer = Input(shape=shape)
    output_layer = Dense(C, activation="softmax")(block1_input_layer) #, kernel_regularizer = l2(10)
    return Model(inputs=block1_input_layer, outputs=output_layer)

In [ ]:
rocketsktime = RocketSktime(shape = (9996,))
rocketsktime.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 9996)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 31)                  │         309,907 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 309,907 (1.18 MB)

 Trainable params: 309,907 (1.18 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
rocketsktime_model = RocketSktime(shape = (9996,))
rocketsktime_model.compile(optimizer=Adam(learning_rate=1.0, beta_1=0.99, beta_2=0.999, epsilon=1e-08), loss='categorical_crossentropy', metrics=['accuracy'])
history_sktime = rocketsktime_model.fit(train_rocket_ds, validation_data=val_rocket_ds, epochs=50, verbose = 1)

Epoch 1/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.0518 - loss: 13221.0527 - val_accuracy: 0.0504 - val_loss: 13282.5947
Epoch 2/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.1223 - loss: 12049.3262 - val_accuracy: 0.1179 - val_loss: 10221.6289
Epoch 3/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.1537 - loss: 8804.2402 - val_accuracy: 0.1673 - val_loss: 5977.6646
Epoch 4/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.2275 - loss: 5126.3472 - val_accuracy: 0.3458 - val_loss: 4285.4326
Epoch 5/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.3132 - loss: 3590.4050 - val_accuracy: 0.3317 - val_loss: 3069.1147
Epoch 6/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.3732 - loss: 2419.4194 - val_accuracy: 0.4335 - val_loss: 2128.5093
Epoch 7/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.4757 - loss: 1603.3622 - val_accuracy: 0.4929 - val_loss: 1241.1154
Epoch 8/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5

In [ ]:
np.mean(np.abs(rocketsktime_model.get_weights()[0]))

20.238832

The average absolute value of the weights is very high, which may contribute to overfitting. We experiment with an L2 regularisation penalty applied.

### **With L2 penalty**

In [ ]:
def RocketSktimeL2(shape):
    block1_input_layer = Input(shape=shape)
    output_layer = Dense(C, activation="softmax", kernel_regularizer = l2(0.01))(block1_input_layer)
    return Model(inputs=block1_input_layer, outputs=output_layer)

In [ ]:
rocketsktimeL2 = RocketSktimeL2(shape = (9996,))
rocketsktimeL2.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 9996)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 31)                  │         309,907 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 309,907 (1.18 MB)

 Trainable params: 309,907 (1.18 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
rocketsktime_modelL2 = RocketSktimeL2(shape = (9996,))
rocketsktime_modelL2.compile(optimizer=Adam(learning_rate=1.0, beta_1=0.99, beta_2=0.999, epsilon=1e-08), loss='categorical_crossentropy', metrics=['accuracy'])
history_sktimeL2 = rocketsktime_modelL2.fit(train_rocket_ds,  validation_data = val_rocket_ds, epochs=50, verbose = 1)

Epoch 1/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.0451 - loss: 9609.8076 - val_accuracy: 0.0796 - val_loss: 9430.8994
Epoch 2/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.0393 - loss: 8301.2871 - val_accuracy: 0.0353 - val_loss: 6221.8452
Epoch 3/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.0465 - loss: 5808.1123 - val_accuracy: 0.0312 - val_loss: 3612.7710
Epoch 4/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.0567 - loss: 3832.7261 - val_accuracy: 0.0232 - val_loss: 3100.2510
Epoch 5/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.0545 - loss: 3103.2803 - val_accuracy: 0.0776 - val_loss: 2616.8306
Epoch 6/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.0518 - loss: 2518.8999 - val_accuracy: 0.0363 - val_loss: 1775.2327
Epoch 7/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.0537 - loss: 2065.2300 - val_accuracy: 0.0524 - val_loss: 2454.2278
Epoch 8/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.0435

For all regularisation penalties tested (0.01, 0.1, 1, 10, 100) the performance was markedly worse than when no regularisation penalty was applied.

## **Experiment: Rocket transformation with ensemble methods**

### **Random Forest**

Random Forest has the ability to perform classification by picking a large number of decision trees and taking the prediction returned by the largest number of decision trees. Each decision tree includes a random subset of the total number of features in the dataset, so that no 2 trees are highly correlated (as they have been trained on different features). This is why Random Forest is a powerful ensembling method.

In [ ]:
# Define the parameter grid
param_grid = {
    'n_estimators': [1000, 1500],
    'max_depth': [4, 5, 6, 7, 8, 9],
}

# Initialize the RandomForestClassifier
rf = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, return_train_score=True)

# Fit GridSearchCV
grid_search.fit(rocket_np, y1s)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [4, 5, 6, 7, 8, 9],
                         'n_estimators': [1000, 1500]},
             return_train_score=True, verbose=2)

In [ ]:
# Get the cv_results_
cv_results = pd.DataFrame(grid_search.cv_results_)

# Select and print specific columns for easier readability
columns_to_display = ['param_n_estimators', 'param_max_depth', 'mean_test_score', 'std_test_score', 'mean_train_score', 'std_train_score', 'rank_test_score']
print(cv_results[columns_to_display])

   param_n_estimators param_max_depth  mean_test_score  std_test_score  \
0                1000               4         0.730683        0.040010   
1                1500               4         0.731925        0.039544   
2                1000               5         0.825839        0.041188   
3                1500               5         0.827329        0.039937   
4                1000               6         0.889689        0.029787   
5                1500               6         0.889689        0.033429   
6                1000               7         0.919503        0.026705   
7                1500               7         0.920000        0.026417   
8                1000               8         0.956273        0.026133   
9                1500               8         0.957019        0.027762   
10               1000               9         0.976149        0.031567   
11               1500               9         0.976398        0.031071   

    mean_train_score  std_train_score

In [ ]:
print(f"Best parameters {grid_search.best_params_}")
print(f"Best score {grid_search.best_score_}")
print(f"Best estimator {grid_search.best_estimator_}")
print(f"Best index {grid_search.best_index_}")

Best parameters {'max_depth': 9, 'n_estimators': 1500}
Best score 0.9763975155279503
Best estimator RandomForestClassifier(max_depth=9, n_estimators=1500, random_state=42)
Best index 11


In [ ]:
# Finally, save to a Pandas dataframe.
cv_results.to_csv("../models/Random Forest/Grid Search Random Forest.csv")

Of all the models tested so far, Random Forest applied to Rocket features has the strongest performance. As the depth increases, using the Grid Search, the performance increased to 97% on the test data (using a Grid Search cross validation approach) and ~100% on the train data.

The performance does not appear to have peaked either; we could expect that increasing the depth could improve the validation performance further, and we experiment with this below.

Increasing the number of sample estimators from 1,000 to 2,000 does not appear to significantly improve the performance of the Random Forest model. It is reasonable to discount exploring using *more* sample estimators, but worth exploring using *fewer* sample estimators.

We fit several more classifiers with `n_estimators` = $[1000, 500, 250, 200, 100]$. The model performance began dropping off only when `n_estimators` = 100, so we use `n_estimators` = 200 and then explore changing `max_depth`.

In [ ]:
best_rf = RandomForestClassifier(n_estimators = 200, criterion = "gini", max_depth = 9, random_state=42)
best_rf.fit(train_rocket_np, y1s_train)
print("Best Random Forest Train Score {}", best_rf.score(train_rocket_np, y1s_train))
print("Best Random Forest Validation Score {}", best_rf.score(val_rocket_np, y1s_val))

Best Random Forest Train Score {} 0.9996702934388394
Best Random Forest Validation Score {} 0.7399193548387096


The performance began dropping off for `max_depth` = 10 and more so for `max_depth` = 11, and we observed from the Grid Search that the performance improved until `max_depth` = 9, so we use `max_depth` = 9.

In [ ]:
higher_depth_rf = RandomForestClassifier(n_estimators = 200, criterion = "gini", max_depth = 11, random_state=42)
higher_depth_rf.fit(train_rocket_np, y1s_train)
print("Higher Depth Random Forest Score {}", higher_depth_rf.score(val_rocket_np, y1s_val))

Higher Depth Random Forest Score {} 0.7167338709677419


### **Adaboost**

Adaboost is an example of a *boosting* method which creates a strong classifier from a large number of weak *base learners.* It starts off with a weak model, but builds successive models based on the errors from earlier models, over time minimising the error. Adaboost starts off with a decision tree base learner with exactly 1 level, known as a decision stump.

Adaboost initially selects a training subset randomly, but iteratively adjusts the weights of training samples so as to assign higher weights to wrongly classified observations for the purposes of correctly classifying them in successive iterations. Moreover, the idea of weightings also applies at the level of the classifier. More accurate classifiers will get a higher weight in the ensemble of models.

In [ ]:
ada = AdaBoostClassifier(n_estimators=100, algorithm="SAMME", random_state=0)
ada.fit(train_rocket_np, y1s_train)
print("AdaBoost Validation Score {}", ada.score(val_rocket_np, y1s_val))

AdaBoost Validation Score {} 0.1975806451612903


There are fewer parameters that can be manipulated with Adaboost compared to Random Forest. Tweaking `n_estimators` in the range $[100, 1000]$ did not make the model competitive with Random Forest.

### **Gradient Boosting**

This method improves on AdaBoost, re-introducing many of the Random Forest parameters.

In [ ]:
gb = GradientBoostingClassifier(n_estimators=200, learning_rate=1.0, max_depth = 9, loss = "log_loss")
gb.fit(train_rocket_np, y1s_train)
print("Gradient Boosting Validation Score {}", gb.score(val_rocket_np, y1s_val))

Gradient Boosting Validation Score {} 0.3538306451612903


Gradient Boosting had amongst the slowest training time of the models tested, so it was not possible to perform a Grid Search over a large number of hyper-parameters. It was competitive with AdaBoost, but not with the best performing model, Random Forest.

## **References**

This workbook is based on the Rocket, MiniRocket and MultiRocket papers:

**Rocket**: Dempster, Angus, Franccois Petitjean and Geoffrey I. Webb. “ROCKET: exceptionally fast and accurate time series classification using random convolutional kernels.” Data Mining and Knowledge Discovery 34 (2019): 1454 - 1495.

**MiniRocket**: Dempster, Angus, Daniel F. Schmidt and Geoffrey I. Webb. “MiniRocket: A Very Fast (Almost) Deterministic Transform for Time Series Classification.” Proceedings of the 27th ACM SIGKDD Conference on Knowledge Discovery & Data Mining (2020): n. pag.

**MultiRocket**: Tan, Chang Wei, Angus Dempster, C. Bergmeir and Geoffrey I. Webb. “MultiRocket: multiple pooling operators and transformations for fast and effective time series classification.” Data Mining and Knowledge Discovery 36 (2021): 1623 - 1646.